# Project IRIS: Sprint 1 - Data Fusion

**Objective:** This notebook performs the final step of Sprint 1: fusing the certified INFOBRAS and SINADEF datasets. The output will be a single, master DataFrame containing both G-Factors and S-Factors at the district level (`ubigeo`), ready for the feature engineering and modeling phases.

In [4]:
import pandas as pd
import numpy as np

print("--- Step 1: Loading Certified Datasets ---")

try:
    # --- LA CORRECCIÓN ESTÁ AQUÍ ---
    # Creamos una lista de las columnas que sabemos que son fechas
    date_columns_infobras = [
        'fecha_de_inicio_de_obra',
        'fecha_de_finalizacion_real',
        'fecha_finalizacion_programada_de_obra',
        'fecha_finalizacion_reprogramada_de_obra'
        # Añade otras columnas de fecha si las vas a usar
    ]

    # Le decimos a Pandas que parsee estas columnas como fechas al momento de la carga
    df_infobras = pd.read_csv(
        '../data/infobras_certificado_v_final.csv', 
        low_memory=False,
        parse_dates=date_columns_infobras # ¡La instrucción clave!
    )
    print(f"Successfully loaded INFOBRAS data: {df_infobras.shape[0]} rows.")

    # Verificamos que los tipos de datos son correctos
    print("\nVerifying dtypes for key date columns in INFOBRAS:")
    print(df_infobras[date_columns_infobras].dtypes)

    # Cargamos los datos de SINADEF (sin cambios)
    df_sinadef = pd.read_csv('../data/sinadef_certified_v1.csv')
    print(f"\nSuccessfully loaded SINADEF data: {df_sinadef.shape[0]} districts.")

except FileNotFoundError as e:
    print(f"CRITICAL ERROR: A required certified file was not found.")
    print(e)

--- Step 1: Loading Certified Datasets ---
Successfully loaded INFOBRAS data: 132137 rows.

Verifying dtypes for key date columns in INFOBRAS:
fecha_de_inicio_de_obra                    datetime64[ns]
fecha_de_finalizacion_real                 datetime64[ns]
fecha_finalizacion_programada_de_obra      datetime64[ns]
fecha_finalizacion_reprogramada_de_obra    datetime64[ns]
dtype: object

Successfully loaded SINADEF data: 2268 districts.


In [6]:
print("\n--- Step 2: Aggregating INFOBRAS Data to District Level (G-Factor Creation) ---")

# Create the 'ubigeo' key in the INFOBRAS dataset
df_infobras['ubigeo'] = df_infobras['departamento'] + '_' + df_infobras['provincia'] + '_' + df_infobras['distrito']

# --- Engineer the G-Factor metrics at the individual project level ---
df_infobras['fue_paralizado'] = (df_infobras['causal_de_paralizacion'] != 'No Paralizada').astype(int)
df_infobras['plazo_real_dias'] = (df_infobras['fecha_de_finalizacion_real'] - df_infobras['fecha_de_inicio_de_obra']).dt.days

# --- CÁLCULOS BLINDADOS ---
# Reemplazamos los ceros en los denominadores con NaN para evitar la división por cero.
# El NaN será ignorado automáticamente por la agregación .mean().

# Blindaje para ratio_sobretiempo
denominador_tiempo = df_infobras['plazo_de_ejecucion_en_dias'].replace(0, np.nan)
df_infobras['ratio_sobretiempo'] = (df_infobras['plazo_real_dias'] - df_infobras['plazo_de_ejecucion_en_dias']) / denominador_tiempo

# Blindaje para ratio_sobrecosto
denominador_costo = df_infobras['monto_viable/aprobado'].replace(0, np.nan)
df_infobras['ratio_sobrecosto'] = (df_infobras['costo_de_la_obra_en_soles'] - df_infobras['monto_viable/aprobado']) / denominador_costo


# --- Aggregate by 'ubigeo' ---
# La función .mean() ignora los valores NaN por defecto, por lo que el promedio será correcto.
infobras_aggregated_df = df_infobras.groupby('ubigeo').agg(
    g_factor_total_obras=('codigo_infobras', 'count'),
    g_factor_tasa_paralizacion=('fue_paralizado', 'mean'),
    g_factor_ratio_sobretiempo_promedio=('ratio_sobretiempo', 'mean'),
    g_factor_ratio_sobrecosto_promedio=('ratio_sobrecosto', 'mean')
).reset_index()

print("INFOBRAS data successfully aggregated by district (with anti-infinity protection).")
display(infobras_aggregated_df.head())


--- Step 2: Aggregating INFOBRAS Data to District Level (G-Factor Creation) ---
INFOBRAS data successfully aggregated by district (with anti-infinity protection).


,ubigeo,g_factor_total_obras,g_factor_tasa_paralizacion,g_factor_ratio_sobretiempo_promedio,g_factor_ratio_sobrecosto_promedio
0,AMAZONAS_BAGUA_ARAMANGO,552,0.065217,0.689552,-0.963116
1,AMAZONAS_BAGUA_BAGUA,118,0.025424,0.888986,-0.964240
2,AMAZONAS_BAGUA_COPALLIN,40,0.000000,0.442342,-1.000000
3,AMAZONAS_BAGUA_EL PARCO,16,0.000000,0.612704,-1.000000
4,AMAZONAS_BAGUA_IMAZA,254,0.118110,1.810063,-0.915809
